















# Financial Data Collection

This notebook downloads historical financial data for the project.

**Data Sources:**
- Stock indices (S&P 500, NASDAQ)
- Individual stocks (AAPL, MSFT, GOOGL)
- Cryptocurrencies (BTC, ETH)

**Time Period:** 2015-01-01 to 2024-12-31 (10 years of daily data)

## 1. Import Libraries

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print("✅ Libraries imported successfully!")

## 2. Configuration

In [ ]:
# Define data parameters
START_DATE = '2015-01-01'
END_DATE = '2024-12-31'

# Define tickers to download
TICKERS = {
    'indices': ['^GSPC', '^IXIC', '^DJI'],  # S&P 500, NASDAQ, Dow Jones
    'stocks': ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA'],  # Tech stocks
    'crypto': ['BTC-USD', 'ETH-USD']  # Cryptocurrencies
}

# Output directory
RAW_DATA_DIR = '../../data/raw'
os.makedirs(RAW_DATA_DIR, exist_ok=True)

print(f"📅 Date Range: {START_DATE} to {END_DATE}")
print(f"📊 Total tickers to download: {sum(len(v) for v in TICKERS.values())}")

## 3. Download Data Function

In [ ]:
def download_ticker_data(ticker, start_date, end_date, save_dir):
    """
    Download historical data for a single ticker and save to CSV.
    
    Parameters:
    -----------
    ticker : str
        Ticker symbol (e.g., 'AAPL', '^GSPC')
    start_date : str
        Start date in 'YYYY-MM-DD' format
    end_date : str
        End date in 'YYYY-MM-DD' format
    save_dir : str
        Directory to save the CSV file
    
    Returns:
    --------
    pd.DataFrame or None
        Downloaded data or None if failed
    """
    try:
        print(f"📥 Downloading {ticker}...", end=' ')
        
        # Download data
        data = yf.download(ticker, start=start_date, end=end_date, progress=False)
        
        if data.empty:
            print("❌ No data available")
            return None
        
        # Clean ticker name for filename (remove special characters)
        clean_ticker = ticker.replace('^', '').replace('-', '_')
        filename = os.path.join(save_dir, f"{clean_ticker}.csv")
        
        # Save to CSV
        data.to_csv(filename)
        
        print(f"✅ Saved ({len(data)} rows) → {filename}")
        return data
        
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        return None

## 4. Download All Data

In [ ]:
# Dictionary to store downloaded data
downloaded_data = {}

# Download each category
for category, tickers in TICKERS.items():
    print(f"\n{'='*60}")
    print(f"📂 Category: {category.upper()}")
    print(f"{'='*60}")
    
    for ticker in tickers:
        data = download_ticker_data(ticker, START_DATE, END_DATE, RAW_DATA_DIR)
        if data is not None:
            downloaded_data[ticker] = data

print(f"\n{'='*60}")
print(f"✅ Download Complete! Successfully downloaded {len(downloaded_data)} tickers")
print(f"{'='*60}")

## 5. Data Summary

In [ ]:
# Create summary dataframe
summary_data = []

for ticker, data in downloaded_data.items():
    summary_data.append({
        'Ticker': ticker,
        'Rows': len(data),
        'Start Date': data.index.min().strftime('%Y-%m-%d'),
        'End Date': data.index.max().strftime('%Y-%m-%d'),
        'Missing Values': data.isnull().sum().sum(),
        'Columns': ', '.join(data.columns)
    })

summary_df = pd.DataFrame(summary_data)
print("\n📊 Data Summary:")
print(summary_df.to_string(index=False))

# Save summary
summary_df.to_csv(os.path.join(RAW_DATA_DIR, '_data_summary.csv'), index=False)
print(f"\n💾 Summary saved to {RAW_DATA_DIR}/_data_summary.csv")

## 6. Quick Visualization

In [ ]:
# Plot closing prices for all downloaded tickers
fig, axes = plt.subplots(3, 1, figsize=(15, 12))

# Plot indices
ax = axes[0]
for ticker in TICKERS['indices']:
    if ticker in downloaded_data:
        data = downloaded_data[ticker]
        ax.plot(data.index, data['Close'], label=ticker, linewidth=2)
ax.set_title('Stock Indices - Closing Prices', fontsize=14, fontweight='bold')
ax.set_ylabel('Price ($)', fontsize=12)
ax.legend(loc='best')
ax.grid(True, alpha=0.3)

# Plot stocks
ax = axes[1]
for ticker in TICKERS['stocks']:
    if ticker in downloaded_data:
        data = downloaded_data[ticker]
        ax.plot(data.index, data['Close'], label=ticker, linewidth=2)
ax.set_title('Individual Stocks - Closing Prices', fontsize=14, fontweight='bold')
ax.set_ylabel('Price ($)', fontsize=12)
ax.legend(loc='best')
ax.grid(True, alpha=0.3)

# Plot crypto
ax = axes[2]
for ticker in TICKERS['crypto']:
    if ticker in downloaded_data:
        data = downloaded_data[ticker]
        ax.plot(data.index, data['Close'], label=ticker, linewidth=2)
ax.set_title('Cryptocurrencies - Closing Prices', fontsize=14, fontweight='bold')
ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Price ($)', fontsize=12)
ax.legend(loc='best')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../../outputs/figures/01_raw_data_overview.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Visualization saved to outputs/figures/01_raw_data_overview.png")

## 7. Sample Data Inspection

In [ ]:
# Display sample data for S&P 500
if '^GSPC' in downloaded_data:
    sp500 = downloaded_data['^GSPC']
    print("📈 S&P 500 Sample Data (First 5 rows):")
    print(sp500.head())
    print("\n📈 S&P 500 Sample Data (Last 5 rows):")
    print(sp500.tail())
    print("\n📊 S&P 500 Statistics:")
    print(sp500.describe())

## 8. Check for Missing Data

In [ ]:
# Check missing values for each ticker
print("🔍 Missing Values Analysis:\n")
for ticker, data in downloaded_data.items():
    missing = data.isnull().sum()
    if missing.sum() > 0:
        print(f"⚠️  {ticker}:")
        print(missing[missing > 0])
        print()
    else:
        print(f"✅ {ticker}: No missing values")

## 9. Next Steps

✅ **Data collection complete!**

**Next notebooks to run:**
1. `02_eda_and_stl.ipynb` - Exploratory Data Analysis and STL Decomposition
2. `03_feature_engineering.ipynb` - Calculate returns, volatility, and other features

**Files created:**
- Raw data CSV files in `data/raw/`
- Data summary in `data/raw/_data_summary.csv`
- Visualization in `outputs/figures/01_raw_data_overview.png`